In [ ]:
import streamlit as st
import plotly.express as px
import pandas as pd
import numpy as np
from openai import OpenAI
import json

# Initialize OpenAI client
# api_key = st.secrets["deepseek"]["api_key"]

client = OpenAI(api_key="sk-19efa6c73dd44494ba316c3bbf34163e", 
                base_url="https://api.deepseek.com")

# Function to fetch parameters from OpenAI API
def fetch_parameters_from_api(country, drug_name, disease):
    prompt = f"""
    You are an expert in pharmacoeconomics. Provide realistic parameters for a pharmacoeconomic analysis of a drug named '{drug_name}' used to treat '{disease}' in '{country}'. Return the response as a JSON object with the following structure, including all parameters needed for the specified analyses. If exact data is unavailable, provide estimated values based on literature or standard practices, and note any assumptions.

    ```json
    {
      "CEA": {
        "C_B": float, // Cost of intervention B (new drug) in USD
        "C_A": float, // Cost of intervention A (comparator) in USD
        "E_B": float, // Effectiveness of B (e.g., life-years gained)
        "E_A": float, // Effectiveness of A (e.g., life-years gained)
        "lambda": float // Willingness-to-pay threshold in USD/unit
      },
      "CUA": {
        "utility_B": [float], // Utility weights for health states (B)
        "time_B": [float], // Time in health states (years, B)
        "utility_A": [float], // Utility weights for health states (A)
        "time_A": [float], // Time in health states (years, A)
        "C_B": float, // Cost of intervention B in USD
        "C_A": float // Cost of intervention A in USD
      },
      "CBA": {
        "benefits_B": float, // Monetary benefits of B in USD
        "benefits_A": float, // Monetary benefits of A in USD
        "C_B": float, // Cost of intervention B in USD
        "C_A": float // Cost of intervention A in USD
      },
      "Decision_Analysis": {
        "probabilities": [float], // Probabilities of outcomes
        "costs": [float], // Costs per outcome in USD
        "lambda": float, // Willingness-to-pay threshold in USD/unit
        "E_B": float, // Effectiveness of B
        "E_A": float, // Effectiveness of A
        "C_B": float, // Cost of intervention B in USD
        "C_A": float // Cost of intervention A in USD
      },
      "Markov_Modeling": {
        "proportions": [float], // Proportions in health states
        "state_costs": [float], // Costs per health state in USD
        "state_outcomes": [float] // Outcomes (e.g., utilities)
      },
      "Discounting": {
        "future_value": float, // Future cost or outcome in USD
        "discount_rate": float, // Discount rate (e.g., 0.03 for 3%)
        "time": float // Time in years
      },
      "Sensitivity_Analysis": {
        "param_min": float, // Minimum cost for sensitivity analysis in USD
        "param_max": float // Maximum cost for sensitivity analysis in USD
      }
    }
    ```

    Ensure all values are realistic for the given context. For example, adjust costs based on '{country}' pricing, and use disease-specific utilities and effectiveness for '{disease}'. Provide at least two health states for CUA and Markov parameters. If assumptions are made, include a brief note in the response.
    """

    try:
        
        response = client.chat.completions.create(
                                                    model="deepseek-chat",
                                                    messages=[
                                                        {"role": "system", "content": "You are a pharmacoeconomic expert providing structured JSON data."},
                                                        {"role": "user", "content": prompt},
                                                    ],
                                                    stream=False,
                                                    temperature=0.3,
                                                    max_tokens=1500
                                                )

        # Parse JSON response
        params = json.loads(response.choices[0].message.content)
        return params
    except Exception as e:
        st.error(f"Ошибка API: {e}. Используются параметры по умолчанию.")
        # Fallback parameters
        return {
            "CEA": {"C_B": 50000, "C_A": 30000, "E_B": 5, "E_A": 3, "lambda": 50000},
            "CUA": {"utility_B": [0.8, 0.6], "time_B": [2, 3], "utility_A": [0.7, 0.5], "time_A": [2, 3], "C_B": 50000, "C_A": 30000},
            "CBA": {"benefits_B": 80000, "benefits_A": 60000, "C_B": 50000, "C_A": 30000},
            "Decision_Analysis": {"probabilities": [0.6, 0.4], "costs": [20000, 40000], "lambda": 50000, "E_B": 5, "E_A": 3, "C_B": 50000, "C_A": 30000},
            "Markov_Modeling": {"proportions": [0.7, 0.3], "state_costs": [15000, 25000], "state_outcomes": [0.8, 0.5]},
            "Discounting": {"future_value": 10000, "discount_rate": 0.03, "time": 5},
            "Sensitivity_Analysis": {"param_min": 20000, "param_max": 60000}
        }